<a href="https://colab.research.google.com/github/Animeshcoder/MySQL-Python/blob/main/Python_MySQL_P5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Introduction:**
This project is a Python script that demonstrates how to search for URLs in a MySQL database. The script defines a search_database function that takes the connection details for a MySQL database as arguments and returns a list of results. This function uses PyMySQL to connect to the database, execute SQL queries, and fetch results. Best part is that you don't need to get the name of table where the URLs are present this code will execute all table and all columns after that it will return all matching rows.


### **Involved Steps:**

**Step 1: Connect to the database**

The search_database function starts by setting up the connection details for the MySQL database and creating a connection object using PyMySQL’s connect function. The host, user, password, and database arguments are passed to this function to specify the connection details.

**Step 2: Query data from the tables**

Next, a cursor object is created using the cursor method of the connection object. This cursor is used to execute an SQL query that retrieves a list of all tables in the database using the SHOW TABLES statement. The table names are stored in a list.

*The function then loops through each table in this list and performs the following steps:*

*Get list of columns:* The code retrieves a list of all columns of type varchar in the current table by executing an SQL query that selects the COLUMN_NAME column from the INFORMATION_SCHEMA.COLUMNS table where the value of the TABLE_NAME column matches the current table name and the value of the DATA_TYPE column is 'varchar'. The column names are stored in a list.

*Build WHERE clause:* The code builds a WHERE clause for an SQL SELECT statement that searches for rows in the current table that contain URLs in any varchar column. This is done by joining together multiple conditions using the OR operator, where each condition uses a regular expression to match columns that contain URLs.

*Execute SELECT statement:* If there are any varchar columns in the current table, the code executes an SQL SELECT statement using the built WHERE clause to search for rows in the current table that contain URLs in any varchar column. Any matching rows are added to a list of results.

**Step 3: Close connection and return results**

After all tables have been searched, both cursor and connection objects are closed by calling their respective close methods. Finally, the list of results is returned to the caller.

In [ ]:
import pymysql

def search_database(host, user, password, database):
    # Set up the connection
    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Create a cursor
    cursor = conn.cursor()

    # Get the list of tables
    cursor.execute("SHOW TABLES")
    tables = [row[0] for row in cursor.fetchall()]

    # Search for the value in all columns of all tables
    results = []
    for table in tables:
        # Get the list of columns
        table_escaped = conn.escape_string(table)
        cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_escaped}' AND DATA_TYPE = 'varchar'")
        columns = [row[0] for row in cursor.fetchall()]

        # Build the WHERE clause
        where_clause = ' OR '.join([f"`{column}` REGEXP 'https?://[^/\\\\s]+' " for column in columns])

        # Check if the where_clause is not empty
        if where_clause:
            # Execute the SELECT statement
            query = f"SELECT * FROM `{table_escaped}` WHERE {where_clause}"
            print(f"Executing query: {query}")
            cursor.execute(query)

            # Fetch the results
            rows = cursor.fetchall()
            if rows:
                results.append((table, rows))

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return results

# Example usage
results = search_database(
    host='yourhost',
    user='youruser',
    password='yourpassword@123',
    database='yourdatabasename'
)

for table, rows in results:
    print(f"Found {len(rows)} match(es) in table '{table}':")
    for row in rows:
        print(row)

# Note: In MySQL queries if your column or table or database name has speces inbetween then then you have to use backtick in place of quotes for example i have used `{table_escaped}` and `{column}`
#  because i don't know about the table names and column names so it can be anything.